In [4]:
from bsf_env import init_spark, init_mariadb_engine,set_spark_verbosity
from bsf_dbutilities import DBUtils
from pyspark.sql import functions as F
import pandas as pd

try:
    spark.stop()
except NameError:
    # SparkSession doesn't exist
    pass

spark = init_spark("bsf_utilities", log_level="WARN", show_progress=False, enable_ui=True,process_option='manual')
engine = init_mariadb_engine()
ingest_ts = spark.sql("SELECT current_timestamp()").collect()[0][0]

[Spark] Started 'bsf_utilities' log_level=WARN (effective=WARN), progress=False


25/09/30 19:24:41 WARN SQLConf: The SQL config 'spark.sql.adaptive.shuffle.targetPostShuffleInputSize' has been deprecated in Spark v3.0 and may be removed in the future. Use 'spark.sql.adaptive.advisoryPartitionSizeInBytes' instead of it.
25/09/30 19:24:41 WARN SQLConf: The SQL config 'spark.sql.adaptive.shuffle.targetPostShuffleInputSize' has been deprecated in Spark v3.0 and may be removed in the future. Use 'spark.sql.adaptive.advisoryPartitionSizeInBytes' instead of it.
25/09/30 19:24:41 WARN SQLConf: The SQL config 'spark.sql.adaptive.shuffle.targetPostShuffleInputSize' has been deprecated in Spark v3.0 and may be removed in the future. Use 'spark.sql.adaptive.advisoryPartitionSizeInBytes' instead of it.


In [6]:
# ─── Setup Database Communications ──────────────────────────────────────
db = DBUtils(spark, ingest_ts)
db.spark_stats()

pd.set_option('display.max_columns', None)  # Show all columns
pd.set_option('display.width', 200)         # Expand width to avoid line breaks


from delta.tables import DeltaTable
import datetime, os
from pyspark.sql import functions as F

# List all databases
databases = [db.name for db in spark.catalog.listDatabases()]

for database in databases:
    print(f"\n=== Database: {database} ===\n")
    
    # Get all tables in this database
    tables = spark.catalog.listTables(database)
    if not tables:
        print("No tables found.")
        continue

    table_info = []
    
    for t in tables:
        full_table_name = f"{database}.{t.name}"
        row_count = spark.table(full_table_name).count()
        table_type = t.tableType
    
        # Default: no max date
        from pyspark.sql import functions as F
        import datetime
        import os
        
        max_date = None
        try:
            df_table = spark.table(full_table_name)
            
            if "LastLoadDate" in df_table.columns:
                max_date = df_table.agg(F.max("LastLoadDate")).collect()[0][0]
                field = "LastLoadDate"
            elif "StockDate" in df_table.columns:
                max_date = df_table.agg(F.max("StockDate")).collect()[0][0]
                field = "StockDate"
            elif "ChangeDate" in df_table.columns:
                max_date = df_table.agg(F.max("ChangeDate")).collect()[0][0]
                field = "ChangeDate"
            else:
                try:
                    dt = DeltaTable.forName(spark, full_table_name)
                    history = dt.history(1)
                    max_date = history.select(F.max("timestamp")).collect()[0][0]
                    field = "timestamp"
                except:
                    max_date = None
        
            # --- Normalize to datetime ---
            if isinstance(max_date, datetime.date) and not isinstance(max_date, datetime.datetime):
                max_date = datetime.datetime.combine(max_date, datetime.time.min)
        
        except:
            # Non-Delta fallback
            table_location = (
                spark.sql(f"DESCRIBE FORMATTED {full_table_name}")
                     .filter("col_name='Location'")
                     .select("data_type")
                     .collect()[0][0]
            )
            max_date = datetime.datetime.fromtimestamp(os.path.getmtime(table_location))


    
        table_info.append((t.name, row_count, field, max_date, table_type))
    
    df_table_info = spark.createDataFrame(
        table_info,
        schema=["TableName", "RowCount","DateField", "MaxDate", "TableType"]
    )
    
    df_table_info.show(truncate=False)
    # 🚨 Safety stop — prevents accidental full execution
    raise RuntimeError("⚠️ This notebook is blocked. Do NOT run all cells without checking!")

    🔍 === Runtime Config Check ===
      📌 User                       : jupyter
      📌 Python Version             : 3.9.21
    🔍 === Spark Runtime Config Check ===
      📌 Name                       : bsf_utilities
      📌 Master                     : local[1]
      📌 Version                    : Not set
      📌 Max Cores                  : 1
      📌 Executor Instances         : 1
      📌 Executor Cores             : 1
      📌 Task Cpus                  : 1
      📌 Executor Memory            : 512m
      📌 Driver Memory              : 512m


25/09/30 19:28:24 WARN SQLConf: The SQL config 'spark.sql.adaptive.shuffle.targetPostShuffleInputSize' has been deprecated in Spark v3.0 and may be removed in the future. Use 'spark.sql.adaptive.advisoryPartitionSizeInBytes' instead of it.



=== Database: bsf ===



25/09/30 19:28:25 WARN SQLConf: The SQL config 'spark.sql.adaptive.shuffle.targetPostShuffleInputSize' has been deprecated in Spark v3.0 and may be removed in the future. Use 'spark.sql.adaptive.advisoryPartitionSizeInBytes' instead of it.


+-----------------------------+--------+----------+--------------------------+---------+
|TableName                    |RowCount|DateField |MaxDate                   |TableType|
+-----------------------------+--------+----------+--------------------------+---------+
|company                      |31051   |ChangeDate|2025-09-30 09:03:07.840831|MANAGED  |
|companyfundamental           |94446   |timestamp |2025-09-30 19:20:21.049   |MANAGED  |
|companystockhistory          |644789  |StockDate |2025-09-29 00:00:00       |MANAGED  |
|companystockhistory_watermark|2104    |timestamp |2025-09-30 19:18:53.368   |MANAGED  |
|history_signal_driver        |644156  |StockDate |2025-09-29 00:00:00       |MANAGED  |
|signaldriver                 |9674    |StockDate |2025-09-24 00:00:00       |MANAGED  |
+-----------------------------+--------+----------+--------------------------+---------+



RuntimeError: ⚠️ This notebook is blocked. Do NOT run all cells without checking!

In [7]:
spark.stop()

In [ ]:
sdf = spark.table("bsf.company")
sdf.toPandas().describe()


In [5]:
from pyspark.sql import functions as F

sdf = spark.table("bsf.companystockhistory_watermark")

# Count rows per timeframe
sdf.groupBy("CompanyId") \
   .agg(F.count("*").alias("row_count")) \
   .orderBy("CompanyId") \
   .show(truncate=False)


25/09/30 19:26:47 WARN SQLConf: The SQL config 'spark.sql.adaptive.shuffle.targetPostShuffleInputSize' has been deprecated in Spark v3.0 and may be removed in the future. Use 'spark.sql.adaptive.advisoryPartitionSizeInBytes' instead of it.
25/09/30 19:26:47 WARN HiveConf: HiveConf of name hive.stats.jdbc.timeout does not exist
25/09/30 19:26:47 WARN HiveConf: HiveConf of name hive.metastore.client.connect.timeout does not exist
25/09/30 19:26:47 WARN HiveConf: HiveConf of name hive.stats.retries.wait does not exist
25/09/30 19:26:47 WARN ObjectStore: Failed to get database global_temp, returning NoSuchObjectException


+---------+---------+
|CompanyId|row_count|
+---------+---------+
|52       |1        |
|91       |1        |
|138      |1        |
|145      |1        |
|151      |1        |
|205      |1        |
|216      |1        |
|253      |1        |
|320      |1        |
|325      |1        |
|332      |1        |
|383      |1        |
|397      |1        |
|424      |1        |
|437      |1        |
|458      |1        |
|471      |1        |
|494      |1        |
|497      |1        |
|521      |1        |
+---------+---------+
only showing top 20 rows



In [ ]:
from delta.tables import DeltaTable
dt = DeltaTable.forName(spark, "bsf.history_signals")
dt.vacuum(retentionHours=0)  # ⚠ deletes unreferenced files immediately


In [ ]:
from delta.tables import DeltaTable
dt = DeltaTable.forName(spark, "bsf.history_signals")
dt.history().show(truncate=False)



In [ ]:
spark.read.format("delta").load("/srv/lakehouse/tables/bsf.db/history_signals") \
     .select("TimeFrame").distinct().show()



In [ ]:
count = spark.sql(f"""
    SELECT COUNT(DISTINCT CompanyId) as cnt
    FROM bsf.company
    WHERE ListingExchange IN (1,2,3,16)
      AND Active = 1
      AND LastClose < 0.1
      AND LastHistoryDate >= date_sub(current_date(), 30)
""").collect()[0]["cnt"]

print(count)




In [ ]:
sdf = spark.table("bsf.companystockhistory")
sdf.toPandas().describe()

In [ ]:
sdf = spark.table("bsf.companystockhistory")
counts_df = (
    sdf.groupBy("CompanyId")
      .agg(F.count("*").alias("row_count"))
      .orderBy("CompanyId")
)

counts_df.show(50, truncate=False)
#pdf.head(10).T

In [ ]:
sdf = spark.table("bsf.companystockhistory_watermark")
pdf = sdf.limit(10).toPandas()
#pdf.head(10).T

In [ ]:
sdf = spark.table("bsf.history_signals_last")
pdf = sdf.limit(10).toPandas()
pdf

In [ ]:
sdf = spark.table("bsf.signaldriver")
sdf.toPandas().describe()



In [ ]:
sdf = spark.table("bsf.history_signals")
pdf = sdf.limit(10).toPandas()
pdf

In [ ]:
sdf = spark.table("bsf.final_candidates")
pdf = sdf.limit(10).toPandas()
pdf

In [ ]:
sdf = spark.table("bsf.company")
# Group by CompanyId, TimeFrame, StockDate
counts_df = (
    sdf.groupBy("ListingExchange")
      .agg(F.count("*").alias("row_count"))
      .orderBy("ListingExchange")
)

counts_df.show(50, truncate=False)



sdf = spark.table("bsf.history_signals")

# Group by CompanyId, TimeFrame, StockDate
counts_df = (
    sdf.groupBy( "TimeFrame")
      .agg(F.count("*").alias("row_count"))
      .orderBy( "TimeFrame")
)

counts_df.show(50, truncate=False)


In [ ]:
# Assume you have a Spark DataFrame
sdf = spark.table("bsf.companystockhistory")

# 1️⃣ Show the schema (data types included)
sdf.printSchema()

# 2️⃣ Get a list of column names
# print(sdf.columns)

# 3️⃣ Show first few rows with all columns (default shows truncated view)
sdf.show(truncate=False)  

# 4️⃣ For more detailed metadata about columns
sdf.dtypes  # Returns a list of (column_name, data_type)


In [ ]:
# 🚨 Safety stop — prevents accidental full execution
raise RuntimeError("⚠️ This notebook is blocked. Do NOT run all cells without checking!")
